In [24]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


**Mounting drive for reading data**

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importing libraries**

In [0]:
import numpy as np
import pandas as pd
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer

**Reading data**

In [0]:
wine_data = pd.read_csv('drive/My Drive/OSX_DS_assignment.csv')

In [28]:
wine_data.shape

(103322, 12)

Removing duplicate records if any from the dataset

In [0]:
wine_data.drop_duplicates(keep='last',inplace=True)

In [30]:
wine_data.shape

(95455, 12)

**Check for missing values**

Removing unwanted features

In [0]:
wine_data.drop('user_name',axis=1,inplace=True)

Identifying & dropping columns containing more than 50% of missing values

In [32]:
total = wine_data.isnull().sum().sort_values(ascending=False)
percent = (wine_data.isnull().sum()/wine_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Missing Percent'])
missing_data['Missing Percent'] = missing_data['Missing Percent'].apply(lambda x: x * 100)
missing_data.loc[missing_data['Missing Percent'] > 0][:10]

,Total,Missing Percent
region_2,53910,56.476874
designation,27363,28.665863
region_1,14804,15.508879
price,6514,6.824158
province,36,0.037714
country,36,0.037714


Dropping region_2 column as it contains more than 50 % of missing values

In [0]:
wine_data.drop('region_2',axis=1,inplace=True)

Change in climate has a effect on wine production.There are 36 records where province is missing.We can drop these records as the number is very less when we compare with total number of records.

In [0]:
wine_data.dropna(how='any',subset=['province','country'],inplace=True)

Resetting index

In [0]:
wine_data.reset_index(drop=True,inplace=True) 

In [36]:
wine_data.head(3)

,country,review_title,review_description,designation,points,price,province,region_1,winery,variety
0,Italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Nicosia,White Blend
1,Portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Quinta dos Avidagos,Portuguese Red
2,US,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Rainstorm,Pinot Gris


Imputing missing prices by the mode price of each country.

In [0]:
wine_data['price'] = wine_data.groupby(['country'])['price'].transform(lambda x: x.fillna(x.mode()[0]))

In [38]:
wine_data.head(3)

,country,review_title,review_description,designation,points,price,province,region_1,winery,variety
0,Italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,20.0,Sicily & Sardinia,Etna,Nicosia,White Blend
1,Portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Quinta dos Avidagos,Portuguese Red
2,US,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Rainstorm,Pinot Gris


In [39]:
total = wine_data.isnull().sum().sort_values(ascending=False)
percent = (wine_data.isnull().sum()/wine_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Missing Percent'])
missing_data['Missing Percent'] = missing_data['Missing Percent'].apply(lambda x: x * 100)
missing_data.loc[missing_data['Missing Percent'] > 0][:10]

,Total,Missing Percent
designation,27359,28.672487
region_1,14768,15.477001


Now we are left with missing values in designation and region_1 column.If we analyze the review title column carefully then we can see that it contains the folloing informations.

*   Year (can be assumed as the production year)
*   Winery name (First part of the title,before year)
*   Province or region_1 (Last part inside parenthesis)

It also contains informations related to designation and variety but it is hard to differentiate as there are more than 27000 records where designation is not available.








**Creating new feature of wine age from the date available in review title column**

In [0]:
import re
import datetime

In [0]:
def find_wine_age(data,col):
  MFG = []
  for i in range(len(data[col])):
    temp = re.findall(r'\b(\d{4})\b',data[col][i])
    designation = data['designation'][i]
    if pd.isnull(designation):
      temp = [j for j in temp if j not in ['']]
    else:
      temp = [j for j in temp if j not in designation]
    year = ''.join([i for i in temp if not i=='' and int(i)>1950 and int(i)<2021])
    MFG.append(year)
  data['MFG'] = pd.Series(MFG)
  data['wine_age'] = data.MFG.apply(lambda x: datetime.date.today().year - int(x) if not x == '' else 0)
  data.drop('MFG',axis=1,inplace=True)
  return data

In [0]:
wine_data = find_wine_age(wine_data,col='review_title')

In [43]:
wine_data.sample(3)

,country,review_title,review_description,designation,points,price,province,region_1,winery,variety,wine_age
65746,France,Château Ladignac 2014 Médoc,"This is tight, juicy wine produced by the loca...",NaN,86,20.0,Bordeaux,Médoc,Château Ladignac,Bordeaux-style Red Blend,6
79845,France,Château d'Arcins 2006 Haut-Médoc,"This tastes lean, the fruit buried deep beneat...",NaN,84,24.0,Bordeaux,Haut-Médoc,Château d'Arcins,Bordeaux-style Red Blend,14
55336,US,Sinor-LaVallee 2013 Black Label Bassi Vineyard...,"This is a heady wine from Mike Sinor, with bla...",Black Label Bassi Vineyard,94,45.0,California,San Luis Obispo County,Sinor-LaVallee,Pinot Noir,7


In [0]:
import pandas_profiling
import warnings
warnings.simplefilter('ignore')

**EDA using pandas profoling**

In [0]:
profile = pandas_profiling.ProfileReport(wine_data)

In [46]:
profile

Number of variables,11
Number of observations,95419
Total Missing (%),4.0%
Total size in memory,8.0 MiB
Average record size in memory,88.0 B
Numeric,3
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0
